# Lab | Web Scraping

Welcome to the "Books to Scrape" Web Scraping Adventure Lab!

**Objective**

In this lab, we will embark on a mission to unearth valuable insights from the data available on Books to Scrape, an online platform showcasing a wide variety of books. As data analyst, you have been tasked with scraping a specific subset of book data from Books to Scrape to assist publishing companies in understanding the landscape of highly-rated books across different genres. Your insights will help shape future book marketing strategies and publishing decisions.

**Background**

In a world where data has become the new currency, businesses are leveraging big data to make informed decisions that drive success and profitability. The publishing industry, much like others, utilizes data analytics to understand market trends, reader preferences, and the performance of books based on factors such as genre, author, and ratings. Books to Scrape serves as a rich source of such data, offering detailed information about a diverse range of books, making it an ideal platform for extracting insights to aid in informed decision-making within the literary world.

**Task**

Your task is to create a Python script using BeautifulSoup and pandas to scrape Books to Scrape book data, focusing on book ratings and genres. The script should be able to filter books with ratings above a certain threshold and in specific genres. Additionally, the script should structure the scraped data in a tabular format using pandas for further analysis.

**Expected Outcome**

A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`. The function should scrape book data from the "Books to Scrape" website and return a `pandas` DataFrame with the following columns:

**Expected Outcome**

- A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`.
- The function should return a DataFrame with the following columns:
  - **UPC**: The Universal Product Code (UPC) of the book.
  - **Title**: The title of the book.
  - **Price (£)**: The price of the book in pounds.
  - **Rating**: The rating of the book (1-5 stars).
  - **Genre**: The genre of the book.
  - **Availability**: Whether the book is in stock or not.
  - **Description**: A brief description or product description of the book (if available).
  
You will execute this script to scrape data for books with a minimum rating of `4.0 and above` and a maximum price of `£20`. 

Remember to experiment with different ratings and prices to ensure your code is versatile and can handle various searches effectively!

**Resources**

- [Beautiful Soup Documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- [Pandas Documentation](https://pandas.pydata.org/pandas-docs/stable/index.html)
- [Books to Scrape](https://books.toscrape.com/)


**Hint**

Your first mission is to familiarize yourself with the **Books to Scrape** website. Navigate to [Books to Scrape](http://books.toscrape.com/) and explore the available books to understand their layout and structure. 

Next, think about how you can set parameters for your data extraction:

- **Minimum Rating**: Focus on books with a rating of 4.0 and above.
- **Maximum Price**: Filter for books priced up to £20.

After reviewing the site, you can construct a plan for scraping relevant data. Pay attention to the details displayed for each book, including the title, price, rating, and availability. This will help you identify the correct HTML elements to target with your scraping script.

Make sure to build your scraping URL and logic based on the patterns you observe in the HTML structure of the book listings!


---

**Best of luck! Immerse yourself in the world of books, and may the data be with you!**

**Important Note**:

In the fast-changing online world, websites often update and change their structures. When you try this lab, the **Books to Scrape** website might differ from what you expect.

If you encounter issues due to these changes, like new rules or obstacles preventing data extraction, don’t worry! Get creative.

You can choose another website that interests you and is suitable for scraping data. Options like Wikipedia, The New York Times, or even library databases are great alternatives. The main goal remains the same: extract useful data and enhance your web scraping skills while exploring a source of information you enjoy. This is your opportunity to practice and adapt to different web environments!

In [5]:
# Import necessary libraries
import pandas as pd
import requests
from bs4 import BeautifulSoup


url = "https://books.toscrape.com/"
response = requests.get(url)
response.encoding = 'utf-8'  # Force correct encoding
soup = BeautifulSoup(response.text, "html.parser")
response


<Response [200]>

In [6]:
# Set filters for data extraction
min_rating = 4
max_price = 20.0

In [7]:
# Helper functions

# Convert rating in string format to number
rating_to_int = {'One': 1, 'Two': 2, 'Three': 3, 'Four': 4, 'Five': 5}

def parse_price(p_text: str) -> float:   # Convert price string '£51.77' to float
    # Remove all non-digit and non-dot characters
    clean_text = ''.join(c for c in p_text if c.isdigit() or c == '.')
    return float(clean_text)

def parse_rating(article_tag) -> int:   # Extract numeric rating from article tag
    p = article_tag.find('p', class_='star-rating')
    for cls in p.get('class', []):
        if cls in rating_to_int:
            return rating_to_int[cls]
    return 0

In [8]:
# Scraping function scrapes books to Scrape and return books
# with rating >= min_rating and price <= max_price, returns a pandas DataFrame.

def scrape_books(min_rating=4, max_price=20.0):
    base_url = "http://books.toscrape.com/catalogue/"
    page_num = 1
    books_data = []

    while True:
        # Construct page URL
        url = f"{base_url}page-{page_num}.html"
        response = requests.get(url)
        
        # Stop if page does not exist
        if response.status_code != 200:
            break
        
        soup = BeautifulSoup(response.text, "html.parser")
        books = soup.find_all("article", class_="product_pod")
        
        for book in books:
            # Extract basic info
            title = book.h3.a["title"]
            price = parse_price(book.find("p", class_="price_color").text)
            rating = parse_rating(book)

            # Filter by rating and price
            if rating >= min_rating and price <= max_price:
                # Go to detail page for extra info
                detail_link = book.h3.a["href"]
                # Adjust relative URL
                detail_url = base_url + detail_link.replace('../../../', '')
                detail_resp = requests.get(detail_url)
                detail_resp.encoding = 'utf-8'
                detail_soup = BeautifulSoup(detail_resp.text, "html.parser")

                # Description
                description_tag = detail_soup.find("div", id="product_description")
                description = ""
                if description_tag:
                    description = description_tag.find_next_sibling("p").text

                # Genre from breadcrumb
                breadcrumb = detail_soup.find("ul", class_="breadcrumb")
                genre = breadcrumb.find_all("li")[2].text.strip()

                # Availability
                availability = detail_soup.find("p", class_="instock availability").text.strip()

                # UPC
                upc_tag = detail_soup.find("th", text="UPC")
                upc = upc_tag.find_next_sibling("td").text.strip() if upc_tag else ""

                # Store data
                books_data.append({
                    "UPC": upc,
                    "Title": title,
                    "Price (£)": price,
                    "Rating": rating,
                    "Genre": genre,
                    "Availability": availability,
                    "Description": description
                })

        page_num += 1  # Move to next page

    # Convert to DataFrame
    df = pd.DataFrame(books_data)
    return df

# Example Usage
if __name__ == "__main__":
    df_books = scrape_books(min_rating=4, max_price=20.0)

    # Show first few results
    print(df_books.head()) 
    
    # Show total number of books found
    print(f"Total books found: {len(df_books)}")

C:\Users\cpall\AppData\Local\Temp\ipykernel_18124\3419929751.py:51: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  upc_tag = detail_soup.find("th", text="UPC")


                UPC                                              Title  \
0  ce6396b0f23f6ecc                                        Set Me Free   
1  6258a1f6a6dcfe50  The Four Agreements: A Practical Guide to Pers...   
2  6be3beb0793a53e7                                     Sophie's World   
3  657fe5ead67a7767            Untitled Collection: Sabbath Poems 2014   
4  51653ef291ab7ddc                                    This One Summer   

   Price (£)  Rating           Genre             Availability  \
0      17.46       5     Young Adult  In stock (19 available)   
1      17.66       5    Spirituality  In stock (18 available)   
2      15.94       5      Philosophy  In stock (18 available)   
3      14.27       4          Poetry  In stock (16 available)   
4      19.49       4  Sequential Art  In stock (16 available)   

                                         Description  
0  Aaron Ledbetter’s future had been planned out ...  
1  In The Four Agreements, don Miguel Ruiz reveal...  